<a href="https://colab.research.google.com/github/ComponentSoftTeam/AI-110/blob/main/2_Transformer_Demo_with_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 2.1 Introduction to the tokenizer and embeddings of GPT-2

In [ ]:
%%capture
%pip install datasets transformers bertviz

In [ ]:
from transformers import pipeline, set_seed, GPT2Tokenizer, GPT2LMHeadModel
from torch import tensor, numel
from bertviz import model_view

set_seed(42)

In [ ]:
generator = pipeline('text-generation', model='gpt2')

generator("Hello, I'm here in Budapest and I", max_length=30, num_return_sequences=3)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "Hello, I'm here in Budapest and I want to tell you that my dear little dear baby is going to live for many years to come.\n"},
 {'generated_text': "Hello, I'm here in Budapest and I'm happy to share with you that this will be my first visit to Europe after all. Europe is not"},
 {'generated_text': 'Hello, I\'m here in Budapest and I can finally finally say thanks to you all," he says in the Hungarian. "It\'s just a sad'}]

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [ ]:
'Erno' in tokenizer.get_vocab()

False

In [ ]:
tokenizer.convert_ids_to_tokens(tokenizer.encode('Hi there. I am Erno, I am your instructor.'))

['Hi',
 'Ġthere',
 '.',
 'ĠI',
 'Ġam',
 'ĠEr',
 'no',
 ',',
 'ĠI',
 'Ġam',
 'Ġyour',
 'Ġinstructor',
 '.']

In [ ]:
tokenizer.encode('Hi there. I am Erno, I am your instructor.')

[17250, 612, 13, 314, 716, 5256, 3919, 11, 314, 716, 534, 21187, 13]

In [ ]:
encoded = tokenizer.encode('Hi there. I am Erno, I am your instructor.', return_tensors='pt')

encoded

tensor([[17250,   612,    13,   314,   716,  5256,  3919,    11,   314,   716,
           534, 21187,    13]])

In [ ]:
encoded.shape

torch.Size([1, 13])

In [ ]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
# Get all of the model's parameters as a list of tuples.
named_params = list(model.named_parameters())

print('The GPT-2 model has {:} different named parameters.\n'.format(len(named_params)))

print('==== Embedding Layer ====\n')
for p in named_params[0:2]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Decoders ====\n')
for p in named_params[3:146]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')
for p in named_params[-2:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))



In [ ]:
model.transformer.wte(encoded)

tensor([[[-0.0679, -0.1280,  0.0666,  ...,  0.0494, -0.0610, -0.0470],
         [-0.0806,  0.0413,  0.0576,  ..., -0.0095, -0.1874, -0.0539],
         [ 0.0466, -0.0113,  0.0283,  ..., -0.0735,  0.0496,  0.0963],
         ...,
         [-0.0756,  0.0461,  0.0550,  ..., -0.0826,  0.0872, -0.0208],
         [ 0.1415, -0.1637, -0.0499,  ...,  0.0580,  0.0695,  0.0071],
         [ 0.0466, -0.0113,  0.0283,  ..., -0.0735,  0.0496,  0.0963]]],
       grad_fn=<EmbeddingBackward0>)

In [ ]:
model.transformer.wpe(tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]))

tensor([[-1.8821e-02, -1.9742e-01,  4.0267e-03,  ..., -4.3044e-02,
          2.8267e-02,  5.4490e-02],
        [ 2.3959e-02, -5.3792e-02, -9.4879e-02,  ...,  3.4170e-02,
          1.0172e-02, -1.5573e-04],
        [ 4.2161e-03, -8.4764e-02,  5.4515e-02,  ...,  1.9745e-02,
          1.9325e-02, -2.1424e-02],
        ...,
        [ 1.6006e-03,  6.2476e-03,  1.0040e-01,  ..., -4.6657e-03,
          9.3994e-04, -5.8468e-03],
        [-3.5615e-03,  1.7494e-02,  1.0676e-01,  ..., -5.4367e-03,
         -7.9653e-04, -5.6959e-03],
        [ 5.9564e-05,  1.7205e-02,  9.6934e-02,  ..., -1.5799e-03,
         -8.6813e-04, -7.8220e-03]], grad_fn=<EmbeddingBackward0>)

In [ ]:
initial_input = model.transformer.wte(encoded) + model.transformer.wpe(tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11,12]).reshape(1, 13))
initial_input


tensor([[[-0.0867, -0.3254,  0.0706,  ...,  0.0063, -0.0328,  0.0075],
         [-0.0566, -0.0125, -0.0373,  ...,  0.0247, -0.1772, -0.0540],
         [ 0.0509, -0.0961,  0.0828,  ..., -0.0538,  0.0689,  0.0749],
         ...,
         [-0.0740,  0.0523,  0.1554,  ..., -0.0873,  0.0881, -0.0266],
         [ 0.1379, -0.1462,  0.0569,  ...,  0.0525,  0.0687,  0.0014],
         [ 0.0467,  0.0059,  0.1253,  ..., -0.0751,  0.0488,  0.0885]]],
       grad_fn=<AddBackward0>)

In [ ]:
initial_input = model.transformer.drop(initial_input) ###
initial_input

tensor([[[-0.0867, -0.3254,  0.0706,  ...,  0.0063, -0.0328,  0.0075],
         [-0.0566, -0.0125, -0.0373,  ...,  0.0247, -0.1772, -0.0540],
         [ 0.0509, -0.0961,  0.0828,  ..., -0.0538,  0.0689,  0.0749],
         ...,
         [-0.0740,  0.0523,  0.1554,  ..., -0.0873,  0.0881, -0.0266],
         [ 0.1379, -0.1462,  0.0569,  ...,  0.0525,  0.0687,  0.0014],
         [ 0.0467,  0.0059,  0.1253,  ..., -0.0751,  0.0488,  0.0885]]],
       grad_fn=<AddBackward0>)

In [ ]:
model.lm_head ###

Linear(in_features=768, out_features=50257, bias=False)

In [ ]:
for module in model.transformer.h:
    initial_input = module(initial_input)[0]

initial_input = model.transformer.ln_f(initial_input)
initial_input

tensor([[[ 0.5575,  0.6999, -0.1813,  ..., -0.1896,  0.3552,  0.0272],
         [ 0.1139,  1.0027,  0.0723,  ..., -0.2575,  0.6447,  0.0353],
         [ 0.0261,  1.2553, -0.0519,  ..., -0.2093,  0.6309,  0.0952],
         ...,
         [-0.3555,  1.3587,  0.2510,  ...,  0.0516,  0.8672,  0.1381],
         [-0.3212,  1.3428,  0.1555,  ...,  0.0195,  0.8809,  0.1051],
         [-0.2976,  1.3649,  0.1778,  ...,  0.0337,  0.7983,  0.1461]]],
       grad_fn=<NativeLayerNormBackward0>)

In [ ]:
(initial_input == model(encoded, output_hidden_states=True).hidden_states[-1]).all()
initial_input

tensor([[[ 0.5575,  0.6999, -0.1813,  ..., -0.1896,  0.3552,  0.0272],
         [ 0.1139,  1.0027,  0.0723,  ..., -0.2575,  0.6447,  0.0353],
         [ 0.0261,  1.2553, -0.0519,  ..., -0.2093,  0.6309,  0.0952],
         ...,
         [-0.3555,  1.3587,  0.2510,  ...,  0.0516,  0.8672,  0.1381],
         [-0.3212,  1.3428,  0.1555,  ...,  0.0195,  0.8809,  0.1051],
         [-0.2976,  1.3649,  0.1778,  ...,  0.0337,  0.7983,  0.1461]]],
       grad_fn=<NativeLayerNormBackward0>)

In [ ]:
total_params = 0
for param in model.parameters():
    total_params += numel(param)

print(f'Number of params: {total_params:,}')

Number of params: 124,439,808


## 2.2 Introduction to GPT-2 masked multi-headed attention

In [ ]:
import torch
import pandas as pd


In [ ]:
phrase = 'My friend was right about this class. It is so fun!' ###
encoded_phrase = tokenizer(phrase, return_tensors='pt')

response = model(**encoded_phrase, output_attentions=True, output_hidden_states=True)

len(response.attentions)

12

In [ ]:
encoded_phrase

{'input_ids': tensor([[3666, 1545,  373,  826,  546,  428, 1398,   13,  632,  318,  523, 1257,
            0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
response.attentions[-1].shape  # From the final decoder

torch.Size([1, 12, 13, 13])

In [ ]:
encoded_phrase['input_ids'].shape

torch.Size([1, 13])

In [ ]:
tokens = tokenizer.convert_ids_to_tokens(encoded_phrase['input_ids'][0]) ###

tokens

['My',
 'Ġfriend',
 'Ġwas',
 'Ġright',
 'Ġabout',
 'Ġthis',
 'Ġclass',
 '.',
 'ĠIt',
 'Ġis',
 'Ġso',
 'Ġfun',
 '!']

In [ ]:
### Layer index 9, head 0. Check out the almost 60% attention the token it is giving to the token class
arr = response.attentions[9][0][0]

n_digits = 3

attention_df = pd.DataFrame((torch.round(arr * 10**n_digits) / (10**n_digits)).detach()).applymap(float)

attention_df.columns = tokens
attention_df.index = tokens

attention_df


,My,Ġfriend,Ġwas,Ġright,Ġabout,Ġthis,Ġclass,.,ĠIt,Ġis,Ġso,Ġfun,!
My,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Ġfriend,0.968,0.032,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Ġwas,0.824,0.145,0.031,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Ġright,0.979,0.008,0.007,0.005,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Ġabout,0.979,0.008,0.004,0.005,0.005,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Ġthis,0.924,0.031,0.007,0.006,0.016,0.016,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Ġclass,0.946,0.005,0.001,0.001,0.001,0.002,0.044,0.000,0.000,0.000,0.000,0.000,0.000
.,0.691,0.013,0.003,0.003,0.002,0.006,0.269,0.013,0.000,0.000,0.000,0.000,0.000
ĠIt,0.318,0.003,0.003,0.003,0.006,0.018,0.599,0.018,0.032,0.000,0.000,0.000,0.000
Ġis,0.331,0.006,0.002,0.002,0.003,0.018,0.533,0.013,0.062,0.030,0.000,0.000,0.000


In [ ]:
tokens = tokenizer.convert_ids_to_tokens(encoded_phrase['input_ids'][0])
model_view(response.attentions, tokens)

<IPython.core.display.Javascript object>

In [ ]:
tokens

['My',
 'Ġfriend',
 'Ġwas',
 'Ġright',
 'Ġabout',
 'Ġthis',
 'Ġclass',
 '.',
 'ĠIt',
 'Ġis',
 'Ġso',
 'Ġfun',
 '!']

In [ ]:
response.hidden_states[-1].shape

torch.Size([1, 13, 768])

In [ ]:
response.logits.shape

torch.Size([1, 13, 50257])

In [ ]:
pd.DataFrame( ###
    zip(tokens, tokenizer.convert_ids_to_tokens(response.logits.argmax(2)[0])),
    columns=['Sequence up until', 'Next token with highest probability']
)

,Sequence up until,Next token with highest probability
0,My,Ċ
1,Ġfriend,","
2,Ġwas,Ġa
3,Ġright,.
4,Ġabout,Ġthat
5,Ġthis,.
6,Ġclass,.
7,.,ĠI
8,ĠIt,'s
9,Ġis,Ġa


In [ ]:
generator(phrase, max_length=20, num_return_sequences=1, do_sample=False)  # greedy search

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'My friend was right about this class. It is so fun! I love it! I love the'}]

In [ ]:
generator(phrase, max_length=20, num_return_sequences=1, do_sample=True)  # greedy search with sampling

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'My friend was right about this class. It is so fun! Everyone is so open and you have'}]

## 7.3 Pre-training GPT

In [ ]:
from transformers import pipeline, set_seed
from torch import tensor

generator = pipeline('text-generation', model='gpt2', tokenizer=tokenizer)
set_seed(0)

In [ ]:
# Bias
generator("The earth is", max_length=10, num_return_sequences=2, num_beams=2, temperature=0.8)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'The earth is a living, breathing thing, and'},
 {'generated_text': 'The earth is a living, breathing organism. It'}]

## 7.4 Few-shot learning

In [ ]:
print(generator("""Sentiment Analysis
Text: I hate it when my phone battery dies.
Sentiment: Negative
###
Text: My day has been really great!
Sentiment: Positive
###
Text: Not a fan when it is cloudy
Sentiment:""", top_k=2, temperature=0.1,  num_beams=2, max_length=55)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Sentiment Analysis
Text: I hate it when my phone battery dies.
Sentiment: Negative
###
Text: My day has been really great!
Sentiment: Positive
###
Text: Not a fan when it is cloudy
Sentiment: Negative



In [ ]:
print(generator("""Question/Answering
C: Google was founded in 1998 by Larry Page and Sergey Brin while they were Ph.D. students at Stanford University in California. Together they own about 14 percent of its shares and control 56 percent of the stockholder voting power through supervoting stock.
Q: When was Google founded?
A: 1998
###
C: Hugging Face is a company which develops social AI-run chatbot applications. It was established in 2016 by Clement Delangue and Julien Chaumond. The company is based in Brooklyn, New York, United States.
Q: What does Hugging Face develop?
A: social AI-run chatbot applications
###
C: The New York Jets are a professional American football team based in the New York metropolitan area. The Jets compete in the National Football League (NFL) as a member club of the league's American Football Conference (AFC) East division.
Q: What division do the Jets play in?
A:""", top_k=2,  max_length=215,  num_beams=2, temperature=0.5)[0]['generated_text'])


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Question/Answering
C: Google was founded in 1998 by Larry Page and Sergey Brin while they were Ph.D. students at Stanford University in California. Together they own about 14 percent of its shares and control 56 percent of the stockholder voting power through supervoting stock.
Q: When was Google founded?
A: 1998
###
C: Hugging Face is a company which develops social AI-run chatbot applications. It was established in 2016 by Clement Delangue and Julien Chaumond. The company is based in Brooklyn, New York, United States.
Q: What does Hugging Face develop?
A: social AI-run chatbot applications
###
C: The New York Jets are a professional American football team based in the New York metropolitan area. The Jets compete in the National Football League (NFL) as a member club of the league's American Football Conference (AFC) East division.
Q: What division do the Jets play in?
A: The AFC East
###
C:


In [ ]:
## Zero Shot Learning

In [ ]:
# Same question as before, with no previous examples ie Zero-shot learning. Still works
print(generator(
    '''Question/Answering
C: The New York Jets are a professional American football team based in the New York metropolitan area. The Jets compete in the National Football League (NFL) as a member club of the league's American Football Conference (AFC) East division.
Q: What division do the Jets play in?
A:''',
    top_k=2, max_length=80,  num_beams=2, temperature=0.5)[0]['generated_text']
)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Question/Answering
C: The New York Jets are a professional American football team based in the New York metropolitan area. The Jets compete in the National Football League (NFL) as a member club of the league's American Football Conference (AFC) East division.
Q: What division do the Jets play in?
A: The Jets play in the AFC East, which is the


In [ ]:
# Zero-shot doesn't work as much with the sentiment analysis example
print(generator("""Sentiment Analysis
Text: This new music video was so good
Sentiment:""", top_k=2, temperature=0.1,  num_beams=2, max_length=55)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Sentiment Analysis
Text: This new music video was so good
Sentiment: I'm so glad you liked it
Sentiment: I'm so glad you liked it
Sentiment: I'm so glad you liked it
Sentiment: I'm so glad you


In [ ]:
# Zero-shot abstractive summarization

In [ ]:
to_summarize = """This training will focus on how the GPT family of models are used for NLP tasks including abstractive text summarization and natural language generation. The training will begin with an introduction to necessary concepts including masked self attention, language models, and transformers and then build on those concepts to introduce the GPT architecture. We will then move into how GPT is used for multiple natural language processing tasks with hands-on examples of using pre-trained GPT-2 models as well as fine-tuning these models on custom corpora.

GPT models are some of the most relevant NLP architectures today and it is closely related to other important NLP deep learning models like BERT. Both of these models are derived from the newly invented transformer architecture and represent an inflection point in how machines process language and context.

The Natural Language Processing with Next-Generation Transformer Architectures series of online trainings provides a comprehensive overview of state-of-the-art natural language processing (NLP) models including GPT and BERT which are derived from the modern attention-driven transformer architecture and the applications these models are used to solve today. All of the trainings in the series blend theory and application through the combination of visual mathematical explanations, straightforward applicable Python examples within hands-on Jupyter notebook demos, and comprehensive case studies featuring modern problems solvable by NLP models. (Note that at any given time, only a subset of these classes will be scheduled and open for registration.)"""

In [ ]:
print(generator(
    f"""Summarization Task:\n{to_summarize}\nTL;DR:""",
    max_length=512, top_k=5, temperature=0.5, no_repeat_ngram_size=2
)[0]['generated_text'])


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Summarization Task:
This training will focus on how the GPT family of models are used for NLP tasks including abstractive text summarization and natural language generation. The training will begin with an introduction to necessary concepts including masked self attention, language models, and transformers and then build on those concepts to introduce the GPT architecture. We will then move into how GPT is used for multiple natural language processing tasks with hands-on examples of using pre-trained GPT-2 models as well as fine-tuning these models on custom corpora.

GPT models are some of the most relevant NLP architectures today and it is closely related to other important NLP deep learning models like BERT. Both of these models are derived from the newly invented transformer architecture and represent an inflection point in how machines process language and context.

The Natural Language Processing with Next-Generation Transformer Architectures series of online trainings provides a